## Volatility Modeling Example

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import datetime
from statsmodels.tsa.stattools import adfuller as adf
import statsmodels.api as sm
import pmdarima
from arch import arch_model
from collections import OrderedDict
import datetime as dt
import itertools

In [12]:
###########Input Files
mkt_data_file='MarketData.csv'

####1. Read in market data file
mkt_data=pd.read_csv(mkt_data_file)
mkt_data['Date']=pd.to_datetime(mkt_data['Date'])

mkt_data.set_index('Date', inplace=True)
mkt_data['S&P 500']=mkt_data['S&P 500'].astype(float)

mkt_data.head()

,S&P 500
Date,
2009-12-31,1115.10
2010-01-04,1132.98
2010-01-05,1136.52
2010-01-06,1137.14
2010-01-07,1141.70


### Compare underlying distribution

In [13]:
def distribution_selection(data):
    """
    evaluate LLF based on underlying distribution
    """
    res_normal = arch_model(data).fit(disp="off")
    res_t = arch_model(data, dist="t").fit(disp="off")
    res_skewt = arch_model(data, dist="skewt").fit(disp="off")
    lls = pd.Series(OrderedDict((("normal", res_normal.loglikelihood),("t", res_t.loglikelihood),
                                 ("skewt", res_skewt.loglikelihood))))
    print(lls)
    params = pd.DataFrame(OrderedDict((("normal", res_normal.params),("t", res_t.params),("skewt", res_skewt.params),)))
    print(params)

In [14]:
mkt_data_shifted = mkt_data['S&P 500'].shift(1)
mkt_data['daily_Return'] = np.log(mkt_data['S&P 500']/mkt_data_shifted) #daily returns
mkt_data_shifted=mkt_data['S&P 500'].shift(5)
mkt_data['weekly_Return'] = np.log(mkt_data['S&P 500']/mkt_data_shifted) #weekly returns
mkt_data_shifted=mkt_data['S&P 500'].shift(22)
mkt_data['monthly_Return'] = np.log(mkt_data['S&P 500']/mkt_data_shifted) #monthly returns
mkt_data.dropna(inplace = True)
mkt_data

,S&P 500,daily_Return,weekly_Return,monthly_Return
Date,,,,
2010-02-03,1097.28,-0.005489,-0.000200,-0.016110
2010-02-04,1063.11,-0.031636,-0.019948,-0.063653
2010-02-05,1066.19,0.002893,-0.007177,-0.063879
2010-02-08,1056.74,-0.008903,-0.030246,-0.073328
2010-02-09,1070.52,0.012956,-0.030179,-0.064374
...,...,...,...,...
2021-05-24,4197.05,0.009863,0.008076,0.014899
2021-05-25,4188.13,-0.002128,0.014502,0.001902
2021-05-26,4195.99,0.001875,0.019325,0.001997


In [19]:
distribution_selection(mkt_data['weekly_Return']*100) #scale returns first and then compare NLL of the 3 distributions

normal   -5334.705295
t        -5328.130269
skewt    -5319.367328
dtype: float64
            normal          t      skewt
alpha[1]  0.500838   0.560082   0.510045
beta[1]   0.499162   0.439918   0.489955
lambda         NaN        NaN  -0.158133
mu        0.527308   0.525295   0.447177
nu             NaN  20.150491  26.729264
omega     0.207815   0.245353   0.207687


##### skewt distribution is the winner based on NLL

In [20]:
#best model based on LLF
results=pd.DataFrame()

#iterate through different returns and volatility models
for item in list(itertools.product(['monthly_Return','weekly_return','daily_return'],['FIGARCH','GARCH','EGARCH'])):
    if item[0] =='daily_Return':
        t=252
        shift='D'
        data_range=148
    elif item[0] == 'weekly_Return':
        t=52
        shift='W-MON'
    else:
        t=12
        shift='M'
        data_range=67
        
    SP_return = mkt_data.resample(shift).ffill().pct_change().dropna() #calculate daily, weekly, or monthly returns

    index = SP_return.index

    end_loc = np.where(index >= "2015-12-31")[0].min() #last data point

    forecasts = {}
    
    if item[1]=='FIGARCH':
        model = arch_model(SP_return['S&P 500'], vol=item[1],  p=1, q=1, dist='skewt', rescale = True)
    else:
        model = arch_model(SP_return['S&P 500'], vol=item[1],  p=2, o=0, q=2, dist='skewt', rescale = True)
    
    for i in range(0,data_range):
        model_fit = model.fit(first_obs=1, last_obs=i + end_loc,update_freq=4)
        var = model_fit.forecast(horizon=1,reindex=False).variance
        
        fcast = var.iloc[0]
        forecasts[fcast.name]=fcast
        df=pd.DataFrame(forecasts).T
        df.columns=[item[0]+item[1]]
    
    print(item)
    print(pd.DataFrame(forecasts).T)
    results=pd.concat([results,df], axis=1)

#results.to_csv(r'results_calibration.csv')

Iteration:      4,   Func. Count:     34,   Neg. LLF: 31.694951628881572
Iteration:      8,   Func. Count:     65,   Neg. LLF: 26.9827188466405
Iteration:     12,   Func. Count:     93,   Neg. LLF: 26.969637148835613
Iteration:     16,   Func. Count:    121,   Neg. LLF: 26.913483680840784
Iteration:     20,   Func. Count:    151,   Neg. LLF: 26.85742618686145
Iteration:     24,   Func. Count:    179,   Neg. LLF: 26.84235106371053
Iteration:     28,   Func. Count:    207,   Neg. LLF: 26.839041729242467
Optimization terminated successfully    (Exit mode 0)
            Current function value: 26.838660031150788
            Iterations: 28
            Function evaluations: 217
            Gradient evaluations: 28
Iteration:      4,   Func. Count:     34,   Neg. LLF: 49.78145894719948
Iteration:      8,   Func. Count:     65,   Neg. LLF: 27.208181062001206
Iteration:     12,   Func. Count:     93,   Neg. LLF: 27.190053203982966
Iteration:     16,   Func. Count:    121,   Neg. LLF: 27.0761482

C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:752: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,


Iteration:      4,   Func. Count:     41,   Neg. LLF: 33.0940835947872
Iteration:      8,   Func. Count:     76,   Neg. LLF: 2264.077267245106
Iteration:     12,   Func. Count:    112,   Neg. LLF: 2538.876682981914
Iteration:     16,   Func. Count:    151,   Neg. LLF: 480.941716227141
Iteration:     20,   Func. Count:    196,   Neg. LLF: 7430.050327951545
Iteration:     24,   Func. Count:    233,   Neg. LLF: 141.93287097642352
Iteration:     28,   Func. Count:    269,   Neg. LLF: 24.709830204259138
Iteration:     32,   Func. Count:    306,   Neg. LLF: 24.39347599756734
Iteration:     36,   Func. Count:    343,   Neg. LLF: 56.1178816454992
Iteration:     40,   Func. Count:    381,   Neg. LLF: 408.2129029551772
Iteration:     44,   Func. Count:    428,   Neg. LLF: 3846.2342602818608
Iteration:     48,   Func. Count:    486,   Neg. LLF: 4576.541429594546
Iteration:     52,   Func. Count:    558,   Neg. LLF: 1833.6591049195756
Inequality constraints incompatible    (Exit mode 4)
          

C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:752: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,


Iteration:     12,   Func. Count:    108,   Neg. LLF: 29.767541150595534
Iteration:     16,   Func. Count:    140,   Neg. LLF: 29.69484033525483
Iteration:     20,   Func. Count:    172,   Neg. LLF: 29.621535299005668
Iteration:     24,   Func. Count:    204,   Neg. LLF: 29.433962489537162
Iteration:     28,   Func. Count:    236,   Neg. LLF: 29.30735913485201
Iteration:     32,   Func. Count:    278,   Neg. LLF: 29.265950244665973
Iteration:     36,   Func. Count:    310,   Neg. LLF: 29.266438588519268
Optimization terminated successfully    (Exit mode 0)
            Current function value: 29.265119071449572
            Iterations: 37
            Function evaluations: 331
            Gradient evaluations: 37
Iteration:      4,   Func. Count:     40,   Neg. LLF: 31.425577895957304
Iteration:      8,   Func. Count:     77,   Neg. LLF: 30.216134832977616
Iteration:     12,   Func. Count:    112,   Neg. LLF: 29.800937686565575
Iteration:     16,   Func. Count:    144,   Neg. LLF: 29.6497

...

Iteration:     48,   Func. Count:    486,   Neg. LLF: 4576.541429594546
Iteration:     52,   Func. Count:    558,   Neg. LLF: 1833.6591049195756
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 608.2574919271832
            Iterations: 55
            Function evaluations: 572
            Gradient evaluations: 53
Iteration:      4,   Func. Count:     41,   Neg. LLF: 31.308414179531077
Iteration:      8,   Func. Count:     76,   Neg. LLF: 29.8429059016016
Iteration:     12,   Func. Count:    108,   Neg. LLF: 29.77185793550597
Iteration:     16,   Func. Count:    140,   Neg. LLF: 29.693382968525032
Iteration:     20,   Func. Count:    172,   Neg. LLF: 29.53763668187916
Iteration:     24,   Func. Count:    204,   Neg. LLF: 29.348978658238885
Iteration:     28,   Func. Count:    236,   Neg. LLF: 29.290324465669354
Optimization terminated successfully    (Exit mode 0)
            Current function value: 29.29032446994532
            Iterations: 29
    

C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:752: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,



Iteration:     16,   Func. Count:    144,   Neg. LLF: 29.649775808022017
Iteration:     20,   Func. Count:    176,   Neg. LLF: 29.56135388220742
Iteration:     24,   Func. Count:    208,   Neg. LLF: 29.530247371711333
Iteration:     28,   Func. Count:    240,   Neg. LLF: 29.47706760758346
Iteration:     32,   Func. Count:    272,   Neg. LLF: 29.307020270990567
Iteration:     36,   Func. Count:    304,   Neg. LLF: 29.24871614120842
Optimization terminated successfully    (Exit mode 0)
            Current function value: 29.24871616879661
            Iterations: 37
            Function evaluations: 322
            Gradient evaluations: 37
Iteration:      4,   Func. Count:     41,   Neg. LLF: 35.26775823056265
Iteration:      8,   Func. Count:     76,   Neg. LLF: 29.5632626395971
Iteration:     12,   Func. Count:    109,   Neg. LLF: 34.251610535211015
Iteration:     16,   Func. Count:    145,   Neg. LLF: 28.902391145416235
Iteration:     20,   Func. Count:    190,   Neg. LLF: 508.9730607

C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:752: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,
C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:752: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,


Iteration:     24,   Func. Count:    233,   Neg. LLF: 141.93287097642352
Iteration:     28,   Func. Count:    269,   Neg. LLF: 24.709830204259138
Iteration:     32,   Func. Count:    306,   Neg. LLF: 24.39347599756734
Iteration:     36,   Func. Count:    343,   Neg. LLF: 56.1178816454992
Iteration:     40,   Func. Count:    381,   Neg. LLF: 408.2129029551772
Iteration:     44,   Func. Count:    428,   Neg. LLF: 3846.2342602818608
Iteration:     48,   Func. Count:    486,   Neg. LLF: 4576.541429594546
Iteration:     52,   Func. Count:    558,   Neg. LLF: 1833.6591049195756
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 608.2574919271832
            Iterations: 55
            Function evaluations: 572
            Gradient evaluations: 53
Iteration:      4,   Func. Count:     41,   Neg. LLF: 31.308414179531077
Iteration:      8,   Func. Count:     76,   Neg. LLF: 29.8429059016016
Iteration:     12,   Func. Count:    108,   Neg. LLF: 29.77185793550